In [4]:
import pandas as pd
import numpy as np
import datetime
import jieba
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['font.sans-serif'] = ['SimHei'] 
matplotlib.rcParams['font.family']='sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False
from chinese_calendar import is_workday
from pyecharts.charts import Bar

In [2]:
login=pd.read_csv('login(ANSI编码).csv',encoding='ANSI')
login
# 没有缺失值

,user_id,login_time,login_place
0,用户3,2018/9/6 9:32,中国广东广州
1,用户3,2018/9/7 9:28,中国广东广州
2,用户3,2018/9/7 9:57,中国广东广州
3,用户3,2018/9/7 10:55,中国广东广州
4,用户3,2018/9/7 12:28,中国广东广州
...,...,...,...
387139,用户44247,2020/6/18 7:41,中国湖北武汉
387140,用户44247,2020/6/18 8:13,中国湖北
387141,用户44248,2020/6/18 9:09,中国天津
387142,用户44249,2020/6/18 9:43,中国北京


In [3]:
login.isna().sum()

user_id        0
login_time     0
login_place    0
dtype: int64

In [4]:
#一天内重复登录的行为我们将视为一次登录即可，以减少数据量

# 方法一
# data1=login['login_time'].str.split(' ',expand=True)[0]
# login=pd.concat([login,data1],axis=1)
# login.rename(columns={0:'日期'},inplace=True)
# del login['login_time']
# login

# 方法二
def str_datetime(df,column_name='login_time'):
    df[column_name]=pd.to_datetime(df[column_name])
    df[column_name]=df[column_name].apply(lambda x:x.strftime('%Y-%m-%d'))
    return df

login=str_datetime(login)
# 将同一日期用户重复登陆行为删除
login=login.drop_duplicates()
login

,user_id,login_time,login_place
0,用户3,2018-09-06,中国广东广州
1,用户3,2018-09-07,中国广东广州
5,用户3,2018-09-10,中国广东广州
8,用户3,2018-09-10,中国北京
10,用户3,2018-09-10,中国广东
...,...,...,...
387139,用户44247,2020-06-18,中国湖北武汉
387140,用户44247,2020-06-18,中国湖北
387141,用户44248,2020-06-18,中国天津
387142,用户44249,2020-06-18,中国北京


In [5]:
# 以最近的时间为基准，计算出用户每一次登录距离现在的时间
login['时间差1']=pd.to_datetime(login['login_time']).max()-pd.to_datetime(login['login_time'])
login

C:\Users\ttyai\AppData\Local\Temp\ipykernel_8908\1138754743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  login['时间差1']=pd.to_datetime(login['login_time']).max()-pd.to_datetime(login['login_time'])


,user_id,login_time,login_place,时间差1
0,用户3,2018-09-06,中国广东广州,651 days
1,用户3,2018-09-07,中国广东广州,650 days
5,用户3,2018-09-10,中国广东广州,647 days
8,用户3,2018-09-10,中国北京,647 days
10,用户3,2018-09-10,中国广东,647 days
...,...,...,...,...
387139,用户44247,2020-06-18,中国湖北武汉,0 days
387140,用户44247,2020-06-18,中国湖北,0 days
387141,用户44248,2020-06-18,中国天津,0 days
387142,用户44249,2020-06-18,中国北京,0 days


In [6]:
# 重新排序
login=login.reset_index()
del login['index']
# # 地区拆分
# # 运行时间很长-而且切词并不准确，很多没有切除干净
# for i in range(login.shape[0]):
#     li=[str(i.word) for i in HanLP.newSegment("crf").seg(login.iloc[i,2])]
# #     li=[word for word in jieba.cut(login.iloc[i,2])]
#     if len(li)==1:
#         if '中国' in login.iloc[i,2]:
#             login.loc[i,'国家']=login.iloc[i,2][0:2]
#             login.loc[i,'省份']=login.iloc[i,2][2:]
#         else:
#             login.loc[i,'国家']=login.iloc[i,2]
#     elif len(li)==2:
#         login.loc[i,'国家']=li[0]
#         login.loc[i,'省份']=li[1]
#     else:
#         login.loc[i,'国家']=li[0]
#         login.loc[i,'省份']=li[1]
#         login.loc[i,'地区']=li[2]
#     if i % 10000 ==0:
#         print(i)
 
# 改进版本
for i in range(login.shape[0]):
    if login.loc[i,'login_place'][0:2]=='中国':
        login.loc[i,'国家']='中国'
        if '黑龙江' in login.loc[i,'login_place']:
            login.loc[i,'省份']='黑龙江'
            if len(login.loc[i,'login_place'])>5:
                login.loc[i,'地区']=login.loc[i,'login_place'][5:]
            else:pass
        if '新疆维吾尔' in login.loc[i,'login_place']:
            login.loc[i,'省份']='新疆维吾尔'
            if len(login.loc[i,'login_place'])>7:
                login.loc[i,'地区']=login.loc[i,'login_place'][7:]
            else:pass
        if '内蒙古' in login.loc[i,'login_place']:
            login.loc[i,'省份']='内蒙古'
            if len(login.loc[i,'login_place'])>5:
                login.loc[i,'地区']=login.loc[i,'login_place'][5:]
            else:pass
        else:
            login.loc[i,'省份']=login.loc[i,'login_place'][2:4]
            login.loc[i,'地区']=login.loc[i,'login_place'][4:]   
    else:
        li=[word for word in jieba.cut(login.iloc[i,2])]
        if len(li)==2:
            login.loc[i,'国家']=li[0]
            login.loc[i,'省份']=li[1]
        else:
            login.loc[i,'国家']=li[0]
    if i%10000==0:
        print(f'{round(i*100/(int(login.shape[0])),2)}%')


0.0%


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ttyai\AppData\Local\Temp\jieba.cache
Loading model cost 0.806 seconds.
Prefix dict has been built successfully.


3.74%
7.47%
11.21%
14.94%
18.68%
22.41%
26.15%
29.88%
33.62%
37.35%
41.09%
44.82%
48.56%
52.29%
56.03%
59.76%
63.5%
67.23%
70.97%
74.7%
78.44%
82.17%
85.91%
89.64%
93.38%
97.11%


In [7]:
login.to_excel('用户地区切割.xlsx')

In [5]:
# 重新读取已经切分好城市的数据
login =pd.read_excel('用户地区切割.xlsx',index_col=0)
login = login[login['login_place'] != '中国']
login

,user_id,login_time,login_place,时间差1,国家,省份,地区
0,用户3,2018-09-06,中国广东广州,651,中国,广东,广州
1,用户3,2018-09-07,中国广东广州,650,中国,广东,广州
2,用户3,2018-09-10,中国广东广州,647,中国,广东,广州
3,用户3,2018-09-10,中国北京,647,中国,北京,NaN
4,用户3,2018-09-10,中国广东,647,中国,广东,NaN
...,...,...,...,...,...,...,...
267720,用户44247,2020-06-18,中国湖北武汉,0,中国,湖北,武汉
267721,用户44247,2020-06-18,中国湖北,0,中国,湖北,NaN
267722,用户44248,2020-06-18,中国天津,0,中国,天津,NaN
267723,用户44249,2020-06-18,中国北京,0,中国,北京,NaN


In [10]:
stu_info=pd.read_csv('study_information(ANSI编码).csv',encoding='ANSI')
stu_info

,user_id,course_id,course_join_time,learn_process,price
0,用户3,课程106,2020/4/21 10:11,0,0.0
1,用户3,课程136,2020/3/5 11:44,100,0.0
2,用户3,课程205,2018/9/10 18:17,6300,0.0
3,用户4,课程26,2020/3/31 10:52,0,319.0
4,用户4,课程34,2020/3/31 10:52,0,299.0
...,...,...,...,...,...
194969,用户44245,课程76,2020/6/17 22:16,0,0.0
194970,用户44246,课程76,2020/6/17 22:36,400,0.0
194971,用户44247,课程19,2020/6/18 7:54,10000,499.0
194972,用户44247,课程32,2020/6/18 7:41,0,499.0


In [10]:
stu_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 7.4+ MB


In [11]:
stu_info=stu_info.drop_duplicates()
stu_info.info()
#没有完全重复数据,但是存在价格缺失数据

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 8.9+ MB


In [12]:
stu_info[stu_info.price.isnull()].course_id.value_counts()

课程51    4011
课程96     227
Name: course_id, dtype: int64

In [13]:
stu_course=stu_info.groupby(['course_id']).agg({'price':['max','min']})
stu_course[(stu_course['price']['max']-stu_course['price']['min'])!=0]
#不存在差异定价,且51和96课程价格为nan，其中51和96课程共计4238条数据，暂时不做处理

price    
            max min
course_id          
课程51        NaN NaN
课程96        NaN NaN

In [14]:
# 将进度转化成为数值型，从而便于计算
stu_info['learn_process']=stu_info['learn_process'].apply(lambda x:int(x.split('width:')[-1].split('%')[0]))
# 对加入时间进行相同操作
stu_info=str_datetime(stu_info,column_name='course_join_time')
stu_info

NameError: name 'str_datetime' is not defined

In [ ]:
import plotly.figure_factory as ff
stu_info_corr = stu_info.corr() # 计算相关系数
z = np.round(stu_info_corr.values,3)   # 可视化热力图的数组
x = list(stu_info_corr.columns.values) # 每个变量的名称
fig = ff.create_annotated_heatmap(z, x = x,y = x,colorscale="Viridis",
                                  showscale = True,     ## 显示颜色条
                                  annotation_text = z)  # 设置显示的数值
fig.update_layout(title="",width=800,height=800)
fig.update_layout(title={"x":0.5,"y":0.9})
fig.show()

# emmm，感觉结果毫无关系

In [ ]:
users=pd.read_csv('users.csv',encoding='gbk')
users

In [ ]:
users.info()

In [ ]:
users=users[~users.user_id.isnull()]
users.info()

In [ ]:
# 将是否填写学校信息设置成为0、1变量，作为后续可能有用的信息
users['是否填写学校信息']=users['school']
users.是否填写学校信息[~users['是否填写学校信息'].isnull()]=1
users.是否填写学校信息[users['是否填写学校信息'].isnull()]=0
users.是否填写学校信息.value_counts()

In [ ]:
keys=login.groupby('user_id').login_time.max().index.tolist()
values=login.groupby('user_id').login_time.max().values.tolist()
login_time={}
for i in range(len(keys)):
    login_time[keys[i]]=values[i]
    
u_2=users[users.recently_logged!='--']
u_1=users[users.recently_logged=='--']
for i in range(u_1.shape[0]):
    if u_1.iloc[i,0] in login_time.keys():
        u_1.iloc[i,2]=pd.to_datetime(login_time[u_1.iloc[i,0]])
    else:
        if pd.to_datetime(u_1.iloc[i,1])+datetime.timedelta(days  = int(u_1.iloc[i,5])/480 )>pd.to_datetime('2020-06-18'):
            u_1.iloc[i,2]=pd.to_datetime('2020-06-18')
            print('修改时间为最新时间')
        else:
            u_1.iloc[i,2]=pd.to_datetime(u_1.iloc[i,1])+datetime.timedelta(days  = int(u_1.iloc[i,5])/480 )        
users=pd.concat([u_1,u_2])

In [ ]:
users['recently_logged'].isna().sum()

In [ ]:
# 还是以最近的时间为基准，计算出用户登陆注册行为的时间差
users = users[~users['recently_logged'].isnull()]
str_datetime(users,column_name='register_time')
str_datetime(users,column_name='recently_logged')
users['register_logged_time']=pd.to_datetime(users['recently_logged'])-pd.to_datetime(users['register_time'])
users['register_now_time']=pd.to_datetime(users['register_time']).max()-pd.to_datetime(users['register_time'])
users['logged_now_time']=pd.to_datetime(users['recently_logged']).max()-pd.to_datetime(users['register_time'])
users

In [ ]:
# 现在加入的班级的数量
users['number_of_classes_now']=users['number_of_classes_join']-users['number_of_classes_out']
users

In [ ]:
def nx_data(df=stu_info,group_name=['course_id','user_id']):
    # 得到共现字典
    user_dic={}  
    stu_info_data=df.groupby(group_name)['course_id'].count().unstack()
    column=stu_info_data.columns.tolist()
    for i in range(stu_info_data.shape[0]):
        user_dic[column[i]]=stu_info_data[stu_info_data[column[i]]==1].index.tolist()
        
    #构造共现矩阵
    course_name=list(set(df['course_id'].values.tolist()))
    course_data=pd.DataFrame(data=np.zeros(shape=(len(course_name),len(course_name))),index=course_name,columns=course_name)
    for value in user_dic.values():
        if len(value)==1:
            pass
        else:
            for i in range(len(value)):
                for j in range(i+1,len(value)):
                    course_data.loc[value[i],value[j]]+=1
    return (user_dic,course_data)

user_dic,course_data=nx_data()

for i,key in enumerate(user_dic.keys()):
    users.loc[i,'选课数量']=len(user_dic[key])
users

In [ ]:
# login=pd.read_excel('用户地区切割.xlsx',index_col=0)
login_1=login.sort_values(by=['user_id','时间差1'])
login_del=login_1.user_id.drop_duplicates()
login_diff=login.iloc[list(login_del.index),:]
users_all=pd.merge(users,login_diff,how='left')
users_all.to_excel('全部信息.xlsx')
users_all=users_all.reset_index()
users_all=users_all.drop(columns=['index'])
users_all

In [ ]:
users_all = users_all.drop(labels='Unnamed: 0', axis = 1)
users_all

In [ ]:
import plotly.figure_factory as ff
users_all_corr = users_all.corr() # 计算相关系数
z = np.round(users_all_corr.values,3)   # 可视化热力图的数组
x = list(users_all_corr.columns.values) # 每个变量的名称
fig = ff.create_annotated_heatmap(z, x = x,y = x,colorscale="Viridis",
                                  showscale = True,     ## 显示颜色条
                                  annotation_text = z)  # 设置显示的数值
fig.update_layout(title="",width=800,height=800)
fig.update_layout(title={"x":0.5,"y":0.9})
fig.show()

# emmm，感觉结果毫无关系

In [ ]:
# login_1=login.sort_values(by=['user_id','时间差1'])
# login_del=login_1.user_id.drop_duplicates()
# list1 = list(login_del.index)
# list2 = list(login.user_id.drop_duplicates().index)
# same_items = set(list1) & set(list2)
# len(same_items)

In [ ]:
country=login[login['国家']!='中国'].国家.value_counts().index.tolist()
country_count=login[login['国家']!='中国'].国家.value_counts().values.tolist()
login.国家.value_counts()

In [ ]:
from pyecharts.charts import Line, Pie, Grid
from pyecharts import options as opts
line = (
    Line()
    #添加x轴数据
    .add_xaxis(country)
    #添加y轴数据
    .add_yaxis("各国分布数",country_count, markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"), #标记最大值
                opts.MarkPointItem(type_="min", name="最小值"), #标记最小值
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ),)
    #折线图标题及位置
    .set_global_opts(title_opts=opts.TitleOpts(title="海外分布", pos_left = "30%"))
)

pie = Pie()
#add第一个参数为标签（一般为空就行）
#第二个参数为整合的数据
#第三个参数center为饼图圆心在的位置
#第四个参数rosetype = ’radius‘ 表示要画玫瑰饼图
#第五个参数为标签配置项，这里选择不显示标签
pie.add('', list(zip(country, country_count)), center = ["75%", "40%"],
        rosetype = "radius", label_opts = opts.LabelOpts(is_show = False))
pie.set_global_opts(
    title_opts = opts.TitleOpts(
        #饼状图标题
        title = '海外各国分布占比',
        #标题位置
        pos_left = "70%",
        title_textstyle_opts = opts.TextStyleOpts(
            #标题颜色
            color = 'black',
            #标题大小
            font_size = '12'
        )
    ),
    #图例配置项
    legend_opts = opts.LegendOpts(
        #是否显示图例
        is_show = True,
        #图例纵向排列
        orient = 'vertical',
        #图例位置
        pos_left = "90%"
    )
)
#radius = [70, 100]表示圆环大小,半径70-100的部分
#后面的表示饼图数据显示为百分比形式
pie.set_series_opts(radius=[70, 100], label_opts=opts.LabelOpts(formatter='{b} : {d}%'))

grid = (
    Grid(
        init_opts=opts.InitOpts(
            #并行图主题为浅色
            theme='light',
            #并行图宽度为1200像素
            width='1200px',
        )
    )
    #添加折线图，不显示边框，折线图高度大小，折线图位置
    .add(line, grid_opts=opts.GridOpts(is_show=False, height='50%', pos_right='55%'))
    #添加饼图，不显示边框，饼图高度大小，饼图位置
    .add(pie, grid_opts=opts.GridOpts(is_show=False, height='50%', pos_left='50%'))
)

#渲染成html文件
grid.render('海外分布统计图.html')

In [ ]:
# login=pd.read_excel('用户地区切割.xlsx',index_col=0)
provice=login[login['国家']=='中国'].省份.value_counts().index.tolist()
provice_count=login[login['国家']=='中国'].省份.value_counts().values.tolist()
#中国省份分布图
bar = (
    Bar()
    #添加x轴数据
    .add_xaxis(provice)
    #添加y轴数据
    .add_yaxis("国内省份" ,provice_count, markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"), #标记最大值
                opts.MarkPointItem(type_="min", name="最小值"), #标记最小值
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ),)
    #柱状图标题,位置,区域缩放配置项(实现移动控制x轴的数量)
    .set_global_opts(title_opts=opts.TitleOpts(title="中国省份分布", pos_left = "20%"),
                     datazoom_opts = opts.DataZoomOpts(is_show = True))
)

bar.render("中国省份分布图.html")

In [ ]:
# 热力图的可视化部分，直接运行会出现空白的问题，因此可以生成html文件
from pyecharts.charts import Map

map = (
     Map()
     .add('',list(zip(provice,provice_count)), 'china') #china-cities指的是maptype
     .set_global_opts(
         title_opts=opts.TitleOpts(title='中国用户占比'),
         visualmap_opts=opts.VisualMapOpts(
             is_show = True,
             min_=min(provice_count),
             max_=max(provice_count),  # num.max()
             )
     )
    .set_series_opts(
            # 标签名称显示，默认为True
            label_opts=opts.LabelOpts(is_show=False, color="blue")
        )
    .render("中国用户占比图.html")
 )

In [43]:
from pyecharts.charts import Pie
pie = Pie()

pie.add('', list(zip(provice[:10], provice_count[:10])), center = ["75%", "40%"],
        rosetype = "radius", label_opts = opts.LabelOpts(is_show = False))
pie.set_global_opts(
    title_opts = opts.TitleOpts(
        #饼状图标题
        title = '各省分布占比',
        #标题位置
        pos_left = "70%",
        title_textstyle_opts = opts.TextStyleOpts(
            #标题颜色
            color = 'black',
            #标题大小
            font_size = '12'
        )
    ),
    #图例配置项
    legend_opts = opts.LegendOpts(
        #是否显示图例
        is_show = True,
        #图例纵向排列
        orient = 'vertical',
        #图例位置
        pos_left = "90%"
    )
)
pie.set_series_opts(radius=[70, 100], label_opts=opts.LabelOpts(formatter='{b} : {d}%'))
pie.render("中国各省分布图.html")

'C:\\Users\\LiuJiacheng\\Desktop\\中国各省分布图.html'

In [81]:
users_all = users_all[~users_all['省份'].isnull()]
users_all['learn_time'] = pd.to_numeric(users_all['learn_time'], errors='coerce')
users_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38289 entries, 1 to 43722
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   Unnamed: 0              38289 non-null  float64        
 1   user_id                 38289 non-null  object         
 2   register_time           38289 non-null  object         
 3   recently_logged         38289 non-null  object         
 4   number_of_classes_join  38289 non-null  float64        
 5   number_of_classes_out   38289 non-null  float64        
 6   learn_time              30989 non-null  float64        
 7   school                  7292 non-null   object         
 8   是否填写学校信息                38289 non-null  object         
 9   register_logged_time    38289 non-null  timedelta64[ns]
 10  register_now_time       38289 non-null  timedelta64[ns]
 11  logged_now_time         38289 non-null  timedelta64[ns]
 12  number_of_classes_now   38289 no

In [80]:
users_all.groupby(['省份']).agg({'learn_time':['sum','mean','count'],'number_of_classes_now':['sum','mean'],'选课数量':['mean']})

learn_time                   number_of_classes_now            \
                sum        mean count                   sum      mean   
省份                                                                      
上海市        87037.75   67.262558  1294                 204.0  0.145196   
云南省        65447.73  165.272045   396                 258.0  0.560870   
内蒙古自治区     33098.16  172.386250   192                 167.0  0.732456   
北京市       126005.48   70.710146  1782                 220.0  0.118153   
台湾省         3924.24   93.434286    42                   6.0  0.136364   
吉林省        58278.62  209.635324   278                 106.0  0.339744   
四川省       174544.46  132.632568  1316                 789.0  0.500000   
天津市        33224.04   90.038049   369                 106.0  0.263682   
宁夏回族自治区    17567.81  161.172569   109                  96.0  0.685714   
安徽省       115854.19  149.296637   776                 519.0  0.532308   
山东省       238221.41  167.879782  1419                 870.0  0.506403   
山西省        93120.45  175.038440   532                 541.0  0.759831   
广东省       998024.31  159.556245  6255                5867.0  0.685557   
广西壮族自治区   170034.84  160.561700  1059                 834.0  0.615044   
新疆维吾尔自治区   17645.18  135.732154   130                  76.0  0.487179   
江苏省       203692.69  129.740567  1570                 684.0  0.376860   
江西省       107955.32  155.555216   694                 438.0  0.530909   
河北省       216346.20  175.891220  1230                 906.0  0.599603   
河南省       195380.47  160.806971  1215                 951.0  0.601138   
浙江省       210408.70  141.403696  1488                 890.0  0.502541   
海南省        23901.45  144.857273   165                 108.0  0.516746   
湖北省       369885.15  168.820242  2191                2139.0  0.783230   
湖南省       196528.69  175.315513  1121                 873.0  0.630325   
澳门特别行政区     1245.10  103.758333    12                   9.0  0.562500   
甘肃省        40488.74  129.771603   312                 239.0  0.633952   
省          86584.39  188.637015   459                 513.0  0.881443   
福建省        85324.12  112.268579   760                 441.0  0.498869   
西藏自治区       1380.05  115.004167    12                  10.0  0.769231   
贵州省       167573.30  185.368695   904                 941.0  0.798134   
辽宁省        50520.37   98.672598   512                 130.0  0.232975   
重庆市       170588.26  193.630261   881                 615.0  0.535248   
陕西省       132275.17  129.427759  1022                 550.0  0.447518   
青海省         7121.08  151.512340    47                  25.0  0.416667   
香港特别行政区     9662.66  175.684727    55                  31.0  0.492063   
黑龙江省       46647.22  119.608256   390                 187.0  0.421171   

              选课数量  
              mean  
省份                  
上海市       1.250000  
云南省       2.142857  
内蒙古自治区         NaN  
北京市       1.000000  
台湾省            NaN  
吉林省       1.000000  
四川省       1.125000  
天津市       1.800000  
宁夏回族自治区   1.000000  
安徽省       1.000000  
山东省       1.333333  
山西省       3.900000  
广东省       1.758621  
广西壮族自治区   2.000000  
新疆维吾尔自治区  1.000000  
江苏省       1.714286  
江西省       1.071429  
河北省       1.727273  
河南省       1.555556  
浙江省       1.230769  
海南省       1.000000  
湖北省       3.750000  
湖南省       1.111111  
澳门特别行政区        NaN  
甘肃省       1.750000  
省         1.000000  
福建省       2.428571  
西藏自治区          NaN  
贵州省       1.000000  
辽宁省       1.000000  
重庆市       1.000000  
陕西省       3.600000  
青海省       1.142857  
香港特别行政区        NaN  
黑龙江省      1.000000

In [132]:
# 根据用户的登录时间进行活跃用户可视化
line1 = (
    Line()
    #添加x轴数据
    .add_xaxis(users_all[users_all['recently_logged']>'2020-01-01'].groupby(by='recently_logged').user_id.count().index.tolist())
    #添加y轴数据
    .add_yaxis("登录人数",users_all[users_all['recently_logged']>'2020-01-01'].groupby(by='recently_logged').user_id.count().values.tolist(),
               markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"), #标记最大值
            ]
        ))
    #折线图标题及位置
    .set_global_opts(title_opts=opts.TitleOpts(title="用户登陆时间可视化", pos_left = "10%"))
)
line1.render("用户登陆时间可视化图.html")

'C:\\Users\\LiuJiacheng\\Desktop\\用户登陆时间可视化图.html'

In [49]:
users_all[users_all['recently_logged']=='2020-06-11'].describe()
# 1/4注册时间到距离最新时间都是7天，考虑是进行了推广活动

,Unnamed: 0,number_of_classes_join,number_of_classes_out,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量,时间差1
count,2550.000000,2550.000000,2550.000000,2550,2550,2550,2550.000000,12.000000,284.000000
mean,2400.291373,1.077255,0.004706,9 days 08:03:57.176470588,16 days 08:03:57.176470588,16 days 08:03:57.176470588,1.072549,1.500000,13.429577
std,4073.743722,0.537996,0.073961,40 days 05:01:50.015628286,40 days 05:01:50.015628286,40 days 05:01:50.015628286,0.526869,1.732051,21.551778
min,227.000000,0.000000,0.000000,0 days 00:00:00,7 days 00:00:00,7 days 00:00:00,0.000000,1.000000,7.000000
25%,867.250000,1.000000,0.000000,0 days 00:00:00,7 days 00:00:00,7 days 00:00:00,1.000000,1.000000,7.000000
50%,1509.500000,1.000000,0.000000,0 days 00:00:00,7 days 00:00:00,7 days 00:00:00,1.000000,1.000000,7.000000
75%,2148.750000,1.000000,0.000000,0 days 00:00:00,7 days 00:00:00,7 days 00:00:00,1.000000,1.000000,7.000000
max,43964.000000,7.000000,2.000000,623 days 00:00:00,630 days 00:00:00,630 days 00:00:00,7.000000,7.000000,111.000000


In [50]:
users_all[users_all['recently_logged']=='2020-06-11']

,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量,login_time,login_place,时间差1,国家,省份,地区
225,227.0,用户44011,2020-06-11,2020-06-11,0.0,0.0,2.1,NaN,0,0 days,7 days,7 days,0.0,1.0,2020-06-11,中国福建泉州,7.0,中国,福建省,泉州
227,229.0,用户44009,2020-06-11,2020-06-11,0.0,0.0,0.33,NaN,0,0 days,7 days,7 days,0.0,7.0,2020-06-11,中国江苏,7.0,中国,江苏省,NaN
228,230.0,用户44008,2020-06-11,2020-06-11,0.0,0.0,21.7,NaN,0,0 days,7 days,7 days,0.0,1.0,2020-06-11,中国广东茂名,7.0,中国,广东省,茂名
230,232.0,用户44006,2020-06-11,2020-06-11,0.0,0.0,0,NaN,0,0 days,7 days,7 days,0.0,1.0,2020-06-11,中国江西宜春,7.0,中国,江西省,宜春
231,233.0,用户43972,2020-06-11,2020-06-11,1.0,0.0,0,桂林电子科技大学,1,0 days,7 days,7 days,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38531,38742.0,用户5260,2019-03-23,2020-06-11,1.0,0.0,"2,498.43",NaN,0,446 days,453 days,453 days,1.0,NaN,2020-06-11,中国广东佛山,7.0,中国,广东省,佛山
40911,41157.0,用户2839,2019-02-21,2020-06-11,2.0,0.0,"3,440.57",NaN,0,476 days,483 days,483 days,2.0,NaN,2020-06-11,中国山西长治,7.0,中国,山西省,长治
43158,43414.0,用户573,2018-10-31,2020-06-11,7.0,0.0,"7,807.80",华南师范大学,1,589 days,596 days,596 days,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43699,43958.0,用户26,2018-09-30,2020-06-11,0.0,0.0,4.35,NaN,0,620 days,627 days,627 days,0.0,NaN,2020-06-11,中国浙江杭州,7.0,中国,浙江省,杭州


In [91]:
#6月11日有一个异常点
#非省份因素
users_all[users_all['recently_logged']=='2020-06-11'].groupby(['省份']).user_id.count()
# 筛选出填写信息差异较大，因此考虑是当时进行学校注册优惠活动？
users_all[users_all['recently_logged']=='2020-06-11'].是否填写学校信息.value_counts()

0    193
1     91
Name: 是否填写学校信息, dtype: int64

In [83]:
# 区分工作日和休息日
for i in range(users_all.shape[0]):
    if i == 0:
        if is_workday(pd.to_datetime(users_all.iloc[i,2])):
            users_all.loc[0,'是否工作日']=1
        else:
            users_all.loc[0,'是否工作日']=0
    else: 
        if is_workday(pd.to_datetime(users_all.iloc[i,2])):
            users_all.iloc[i,-1]=1
        else:
            users_all.iloc[i,-1]=0

In [53]:
users_all.是否工作日.value_counts()
# 可以看出工作日登录的时间较少，周末登录时间占比相对较高

1.0    34064
0.0     9660
Name: 是否工作日, dtype: int64

In [54]:
users_all.groupby(['是否填写学校信息','是否工作日']).user_id.count()

是否填写学校信息  是否工作日
0         0.0       8279
          1.0      24905
1         0.0       1381
          1.0       9159
Name: user_id, dtype: int64

In [117]:
ar=[str(aa)[:-14] for aa in users_all.groupby(['logged_now_time']).logged_now_time.count().index.tolist()]
kr=users_all.groupby(['logged_now_time']).logged_now_time.count().values.tolist()

In [133]:
# 用户流失情况分析
# line1=Line()
# line1.add('',ar[:],kr[:],mark_line=["average"])

# line2=Line()
# line2.add('',ar[:140],kr[:140],mark_line=["average"])

# line3=Line('')
# line3.add('',ar[:30],kr[:30],mark_line=["average"])

# line4=Line('')
# line4.add('',ar[:10],kr[:10],mark_line=["average"])
line1 = (
    Line()
    #添加x轴数据
    .add_xaxis(ar[:])
    #添加y轴数据
    .add_yaxis("",kr[:], 
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ))
    #折线图标题及位置
    .set_global_opts(title_opts=opts.TitleOpts(title="用户流失情况分析", pos_left = "30%"))
)
line2 = (
    Line()
    #添加x轴数据
    .add_xaxis(ar[:140])
    #添加y轴数据
    .add_yaxis("",kr[:140], 
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ))
    #折线图标题及位置
    .set_global_opts(title_opts=opts.TitleOpts(title="用户流失情况分析", pos_left = "30%"))
)
line3 = (
    Line()
    #添加x轴数据
    .add_xaxis(ar[:30])
    #添加y轴数据
    .add_yaxis("",kr[:30], 
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ))
    #折线图标题及位置
    .set_global_opts(title_opts=opts.TitleOpts(title="用户流失情况分析", pos_left = "30%"))
)
line4 = (
    Line()
    #添加x轴数据
    .add_xaxis(ar[:10])
    #添加y轴数据
    .add_yaxis("",kr[:10], 
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ))
    #折线图标题及位置
    .set_global_opts(title_opts=opts.TitleOpts(title="用户流失情况分析", pos_left = "30%"))
)

# grid = Grid()
# grid.add(line1, grid_left="55%",grid_top='55%')
# grid.add(line2, grid_right="55%",grid_top='55%')
# grid.add(line3, grid_left="55%",grid_bottom='55%')
# grid.add(line4, grid_right="55%",grid_bottom='55%')
grid = (
    Grid(
        init_opts=opts.InitOpts(
            #并行图主题为浅色
            theme='light',
            #并行图宽度为1200像素
            width='1200px',
        )
    )
    #添加折线图，不显示边框，折线图高度大小，折线图位置
    .add(line1, grid_opts=opts.GridOpts(is_show=False, pos_left="55%",pos_top='55%'))
    .add(line2, grid_opts=opts.GridOpts(is_show=False, pos_right="55%",pos_top='55%'))
    .add(line3, grid_opts=opts.GridOpts(is_show=False, pos_left="55%",pos_bottom='55%'))
    .add(line4, grid_opts=opts.GridOpts(is_show=False, pos_right="55%",pos_bottom='55%'))
)
grid.render('用户流失情况分析.html')

'C:\\Users\\LiuJiacheng\\Desktop\\用户流失情况分析.html'

In [56]:
# 省份流失情况分析
users_all.groupby(['省份','logged_now_time']).user_id.count().unstack()
#对流失时间进行划分

logged_now_time,2 days,4 days,7 days,8 days,9 days,11 days,12 days,14 days,15 days,19 days,...,647 days,653 days,562 days,571 days,612 days,544 days,559 days,613 days,615 days,566 days
省份,,,,,,,,,,,,,,,,,,,,,
上海市,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
云南省,1.0,1.0,NaN,NaN,2.0,1.0,2.0,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
内蒙古自治区,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
北京市,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
台湾省,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
吉林省,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
四川省,1.0,1.0,1.0,1.0,NaN,1.0,2.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
天津市,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
宁夏回族自治区,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
users_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38290 entries, 1 to 0
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   Unnamed: 0              38289 non-null  float64        
 1   user_id                 38289 non-null  object         
 2   register_time           38289 non-null  object         
 3   recently_logged         38289 non-null  object         
 4   number_of_classes_join  38289 non-null  float64        
 5   number_of_classes_out   38289 non-null  float64        
 6   learn_time              30989 non-null  float64        
 7   school                  7292 non-null   object         
 8   是否填写学校信息                38289 non-null  object         
 9   register_logged_time    38289 non-null  timedelta64[ns]
 10  register_now_time       38289 non-null  timedelta64[ns]
 11  logged_now_time         38289 non-null  timedelta64[ns]
 12  number_of_classes_now   38289 non-nu

In [108]:
users_all['logged_now_time'][ : 14]

0    0 days
1    0 days
2    0 days
3    0 days
4    0 days
5    1 days
6    1 days
7    1 days
8    1 days
9    1 days
10   1 days
11   1 days
12   1 days
13   1 days
Name: logged_now_time, dtype: timedelta64[ns]

In [110]:
# for i in range(users_all.shape[0]):
#     if int(str(users_all.loc[i,'logged_now_time'])[:-14]) > 150:        
#         users_all.loc[i,'流失时间划分']='大于150天'
#     elif 90 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 150:        
#         users_all.loc[i,'流失时间划分']='大于90天'
#     elif 30 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 90:        
#         users_all.loc[i,'流失时间划分']='大于30天'
#     elif 15 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 30:        
#         users_all.loc[i,'流失时间划分']='大于15天'
#     elif 7 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 15:        
#         users_all.loc[i,'流失时间划分']='大于7天'
#     elif 0 <= int(str(users_all.loc[i,'logged_now_time'])[:-14]) < 7:        
#         users_all.loc[i,'流失时间划分']='7天内'
users_all

,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,...,logged_now_time,number_of_classes_now,选课数量,login_time,login_place,时间差1,国家,省份,地区,流失时间划分
0,0.0,用户44251,2020-06-18,2020-06-18,0.0,0.0,41.25,NaN,0,0 days,...,0 days,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,7天内
1,1.0,用户44250,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,...,0 days,0.0,1.0,2020-06-18,中国江西南昌,0.0,中国,江西省,南昌,7天内
2,2.0,用户44249,2020-06-18,2020-06-18,0.0,0.0,16.22,NaN,0,0 days,...,0 days,0.0,1.0,2020-06-18,中国北京,0.0,中国,北京市,NaN,7天内
3,3.0,用户44248,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,...,0 days,0.0,1.0,2020-06-18,中国天津,0.0,中国,天津市,NaN,7天内
4,4.0,用户44247,2020-06-18,2020-06-18,0.0,0.0,1.8,NaN,0,0 days,...,0 days,0.0,1.0,2020-06-18,中国湖北武汉,0.0,中国,湖北省,武汉,7天内
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43721,43980.0,用户4,2018-09-10,2020-06-05,4.0,0.0,341.2,NaN,0,634 days,...,647 days,4.0,NaN,2020-06-05,中国广东广州,13.0,中国,广东省,广州,大于150天
43722,43981.0,用户3,2018-09-04,2020-06-18,2.0,1.0,370.35,NaN,0,653 days,...,653 days,1.0,NaN,2020-06-18,中国广东广州,0.0,中国,广东省,广州,大于150天
43723,43982.0,用户1,2018-09-03,2018-11-04,0.0,0.0,0,NaN,0,62 days,...,654 days,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,大于150天
43724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaT,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
users_all.groupby(['省份','流失时间划分']).user_id.count().unstack()

流失时间划分,7天内,大于150天,大于15天,大于30天,大于7天,大于90天
省份,,,,,,
上海市,4.0,1160.0,23.0,93.0,6.0,119.0
云南省,7.0,191.0,8.0,154.0,6.0,94.0
内蒙古自治区,NaN,74.0,6.0,30.0,1.0,117.0
北京市,5.0,1580.0,8.0,118.0,6.0,145.0
台湾省,NaN,33.0,NaN,7.0,1.0,3.0
吉林省,3.0,182.0,3.0,31.0,2.0,91.0
四川省,8.0,886.0,14.0,204.0,9.0,456.0
天津市,5.0,296.0,3.0,31.0,1.0,66.0
宁夏回族自治区,3.0,39.0,19.0,20.0,1.0,58.0


In [59]:
users_all.groupby(['流失时间划分']).agg(['mean']).T

C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_4108\2783475565.py:1: FutureWarning: ['user_id', 'register_time', 'recently_logged', 'learn_time', 'school', 'login_time', 'login_place', '国家', '省份', '地区'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  users_all.groupby(['流失时间划分']).agg(['mean']).T


,流失时间划分,7天内,大于150天,大于15天,大于30天,大于7天,大于90天
Unnamed: 0,mean,113.528889,33180.204957,3173.230606,6540.976132,1452.0,15910.697806
number_of_classes_join,mean,0.484444,0.169967,0.635494,0.940592,0.934313,1.111249
number_of_classes_out,mean,0.022222,0.009924,0.059511,0.016899,0.002448,0.042096
是否填写学校信息,mean,0.031111,0.032148,0.215728,0.176655,0.906161,0.496473
register_logged_time,mean,1 days 19:05:36,44 days 00:45:21.259842519,8 days 11:24:02.295430393,17 days 19:51:38.257839721,0 days 02:16:18.212974296,36 days 19:32:51.222575393
register_now_time,mean,3 days 21:26:24,371 days 12:40:53.561943812,21 days 13:05:02.231668437,61 days 00:22:34.703832753,7 days 07:20:02.937576499,114 days 17:38:52.366850144
logged_now_time,mean,3 days 21:26:24,371 days 12:40:53.561943812,21 days 13:05:02.231668437,61 days 00:22:34.703832753,7 days 07:20:02.937576499,114 days 17:38:52.366850144
number_of_classes_now,mean,0.462222,0.160043,0.575983,0.923693,0.931865,1.069153
选课数量,mean,1.795556,NaN,NaN,NaN,1.642857,NaN
时间差1,mean,2.086538,320.662788,13.310023,42.444381,9.315315,73.474523


In [111]:
# 选课人数最多课程
# bar1=Bar()
# bar1.add('',stu_info.course_id.value_counts().index.tolist()[:30],stu_info.course_id.value_counts().values.tolist()[:30]
#          ,mark_line=['average']
#          ,mark_point=['max','min']
#          , is_datazoom_show=True)
# bar1
provice=login[login['国家']=='中国'].省份.value_counts().index.tolist()
provice_count=login[login['国家']=='中国'].省份.value_counts().values.tolist()

bar = (
    Bar()
    #添加x轴数据
    .add_xaxis(stu_info.course_id.value_counts().index.tolist()[:30])
    #添加y轴数据
    .add_yaxis("人数" ,stu_info.course_id.value_counts().values.tolist()[:30], markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"), #标记最大值
                opts.MarkPointItem(type_="min", name="最小值"), #标记最小值
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ),)
    #柱状图标题,位置,区域缩放配置项(实现移动控制x轴的数量)
    .set_global_opts(title_opts=opts.TitleOpts(title="选课人数最多课程", pos_left = "20%"),
                     datazoom_opts = opts.DataZoomOpts(is_show = True))
)

bar.render("选课人数最多课程.html")

'C:\\Users\\LiuJiacheng\\Desktop\\选课人数最多课程.html'

In [112]:
# 最受欢迎免费课程
# bar1=Bar()
# bar1.add('',stu_info[stu_info['price']==0].course_id.value_counts().index.tolist()[:30]
#          ,stu_info[stu_info['price']==0].course_id.value_counts().values.tolist()[:30]
#         ,mark_line=['average']
#          ,mark_point=['max','min']
#          , is_datazoom_show=True)
bar = (
    Bar()
    #添加x轴数据
    .add_xaxis(stu_info[stu_info['price']==0].course_id.value_counts().index.tolist()[:30])
    #添加y轴数据
    .add_yaxis("人数" ,stu_info[stu_info['price']==0].course_id.value_counts().values.tolist()[:30], markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"), #标记最大值
                opts.MarkPointItem(type_="min", name="最小值"), #标记最小值
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ),)
    #柱状图标题,位置,区域缩放配置项(实现移动控制x轴的数量)
    .set_global_opts(title_opts=opts.TitleOpts(title="最受欢迎免费课程", pos_left = "20%"),
                     datazoom_opts = opts.DataZoomOpts(is_show = True))
)

bar.render("最受欢迎免费课程.html")

'C:\\Users\\LiuJiacheng\\Desktop\\最受欢迎免费课程.html'

In [113]:
# 最受欢迎收费课程
# bar1=Bar()
# bar1.add('',stu_info[stu_info['price']!=0].course_id.value_counts().index.tolist()[:15],stu_info[stu_info['price']!=0].course_id.value_counts().values.tolist()[:15]
#         ,mark_line=['average']
#          ,mark_point=['max','min']
#          , is_datazoom_show=True)
# login=pd.read_excel('用户地区切割.xlsx',index_col=0)

bar = (
    Bar()
    #添加x轴数据
    .add_xaxis(stu_info[stu_info['price']!=0].course_id.value_counts().index.tolist()[:15])
    #添加y轴数据
    .add_yaxis("人数" ,stu_info[stu_info['price']!=0].course_id.value_counts().values.tolist()[:15], markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"), #标记最大值
                opts.MarkPointItem(type_="min", name="最小值"), #标记最小值
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")] #标记平均值线
        ),)
    #柱状图标题,位置,区域缩放配置项(实现移动控制x轴的数量)
    .set_global_opts(title_opts=opts.TitleOpts(title="最受欢迎收费课程", pos_left = "20%"),
                     datazoom_opts = opts.DataZoomOpts(is_show = True))
)

bar.render("最受欢迎收费课程.html")

'C:\\Users\\LiuJiacheng\\Desktop\\最受欢迎收费课程.html'

In [60]:
# 可以看出课程的价格为109时候，用户学习时间和数量都相对较优，但是随着价格的上升，用户学习时间突然下降
stu_info.groupby(['price']).agg({'learn_process':['sum','mean'],'user_id':['count']})

learn_process            user_id
                 sum       mean   count
price                                  
0.0           987006  13.884284   71088
29.0            1460   8.156425     179
49.0            7310  39.728261     184
59.0            1005   5.345745     188
79.0           21325  20.663760    1032
99.0           19474  12.871117    1513
109.0         431260  37.989782   11352
129.0          57465   2.904180   19787
159.0          11302  15.653740     722
169.0          10379  64.067901     162
179.0          24735   8.101867    3053
199.0         479444  14.919218   32136
219.0           2540  10.495868     242
229.0             72   9.000000       8
299.0         461420  15.545972   29681
319.0          33768   6.971098    4844
369.0          33872  44.046814     769
399.0           5080   8.552189     594
499.0          83560  10.835062    7712
600.0          37750  44.940476     840
700.0          28433  84.622024     336
800.0           2000  71.428571      28
899.0          40352  35.678161    1131
999.0              0   0.000000     222
1299.0          6050  55.000000     110
3000.0          5670   2.008502    2823

In [61]:
stu_info[stu_info['price']==129].groupby(['course_id']).agg({'learn_process':['mean','count']})

learn_process      
                   mean count
course_id                    
课程144          1.043697  3982
课程148          1.492337  3719
课程156         38.299120   341
课程158          4.920137  3794
课程162          1.357497  3628
课程163          2.243669  4067
课程45           7.742188   256

In [62]:
stu_info[stu_info['price']==109].groupby(['course_id']).agg({'learn_process':['mean','count']})
#可以看出在价格为109的时候，选择人数和课程学习时间均较长

learn_process      
                   mean count
course_id                    
课程11          81.209302    43
课程30           0.000000     2
课程31          38.663061  9521
课程66          34.009085  1541
课程85          29.587755   245

In [63]:
stu_info[stu_info['price']==299].groupby(['course_id']).agg({'learn_process':['mean','count']})

learn_process      
                   mean count
course_id                    
课程17          28.257613  8505
课程171          3.847526  5437
课程18          87.860465    43
课程34          15.903661  5709
课程35          85.674419    43
课程46           8.186813    91
课程48           4.834513  4520
课程56           3.592782  2743
课程65          30.180971  1503
课程67          22.171113  1087

In [64]:
stu_info[stu_info['price']==369].groupby(['course_id']).agg({'learn_process':['mean','count']})

learn_process      
                   mean count
course_id                    
课程4           44.046814   769

In [65]:
stu_info_course=pd.DataFrame(stu_info.groupby(['course_id']).agg({'learn_process':['mean'],'price':['mean']}))
stu_info_course
# 可以将没有人学习的，或者是相对较少人学习的课程进行删除，而学习时长较长的课程为用户认为较为优质的课程，可以考虑后续重点推广

,learn_process,price
,mean,mean
course_id,,
课程0,0.000000,199.0
课程1,0.000000,199.0
课程10,0.000000,0.0
课程100,0.000000,199.0
课程101,53.475104,0.0
...,...,...
课程95,5.565217,499.0
课程96,7.224670,NaN
